In [1]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Layer

In [4]:
class IdentityBlock(Layer):
  def __init__(self, filters, kernel_size):
    super(IdentityBlock, self).__init__(name ='')

    self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
    self.bn1 = tf.keras.layers.BatchNormalization()

    self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
    self.bn2 = tf.keras.layers.BatchNormalization()

    self.act = tf.keras.layers.Activation('relu')
    self.add = tf.keras.layers.Add()

  def call(self, input_tensor):
    x = self.conv1(input_tensor)
    x = self.bn1(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.add([x, input_tensor])
    x = self.act(x)

    return x

In [10]:
class ResNet(tf.keras.Model):
  def __init__(self, num_classes, activation='relu'):
    super(ResNet, self).__init__()
    self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
    self.bn = tf.keras.layers.BatchNormalization()
    self.act = tf.keras.layers.Activation(activation)
    self.max_pool = tf.keras.layers.MaxPool2D((3, 3))

    self.IdB1 = IdentityBlock(64, 3)
    self.IdB2 = IdentityBlock(64, 3)

    self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
    self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

  def call(self, inputs):
    x = self.conv(inputs)
    x = self.bn(x)
    x = self.act(x)
    x = self.max_pool(x)
    x = self.IdB1(x)
    x = self.IdB2(x)
    x = self.global_pool(x)
    x = self.classifier(x)
    return x

In [11]:
def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']

resnet = ResNet(10)
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
dataset = dataset.map(preprocess).batch(32)
resnet.fit(dataset, epochs=1)

1875/1875 [==============================] - 234s 125ms/step - loss: 0.1573 - accuracy: 0.9588
